In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.utils.data_utils import get_file
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import SimpleRNN, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D
from keras.utils import np_utils
from keras.optimizers import Adam
import pickle
import re
from numpy.random import random, permutation, randn, normal, uniform, choice

C:\Users\Akarsh Somani\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print(len(text))

600901


In [4]:
text[:1000]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--IF,\nindeed, it stands at all! For there are scoffers who maintain that it\nhas fallen, that all dogma lies on the ground--nay more, that it is at\nits last gasp. But to speak seriously, there are good grounds for hoping\nthat all dogmatizing in philosophy, whatever solemn, whatever conclusive\nand decided airs it has assumed, may have been only a noble puerilism\nand tyronism; and probably the time is at hand when it will be once\nand again understood WHAT has actually sufficed for the basis of such\

Creating a vocabulary of unique characters

In [5]:
chars = sorted(list(set(text)))
print(len(chars)+1)

86


In [6]:
chars

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '¤',
 '¦',
 '©',
 '«',
 'Ã',
 '†']

Inserting 0 as it wasn't in the original text

In [7]:
chars.insert(0, '\0')

Creating a dictionary, mapping characters to index and index to characters

In [8]:
char_to_index = {v:i for i,v in enumerate(chars)}
index_to_char = {i:v for i,v in enumerate(chars)}

Converting the entire nietzsche text into index of characters

In [9]:
total_index = [char_to_index[char] for char in text]

In [10]:
total_index[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [11]:
''.join(index_to_char[i] for i in total_index[:25])

'PREFACE\n\n\nSUPPOSING that '

As were are predicting the 8th character, we need to create an array of the first 7 characters each acting as an input and the last character as the output.

For example, for the text 'this and that'

The input will be -> [['t', ' '], ['h', 't'], ['i', 'h'], ['s', 'a'], [' ', 't'], ['a'], ['n']] -> but instead of the characters, there will be the index of the character.

And the output will be -> ['d']

```python
tot='happy birthday my dear friend. hope a good year ahead.'
print('length of text - ',len(tot))
xin = []
y=[]
pred_num=10
count=0
for i in range(pred_num):
  x=[]
  for j in range(0, len(tot)-1-pred_num, pred_num):  
    x.append(tot[j+i])
  print(x)
  xin.append(x)
for i in range(0, len(tot)-1-pred_num, pred_num):  
  y.append(tot[i+pred_num])
print('y=','\n',y)

length of text -  54
['h', 'h', 'a', ' ', 'o']
['a', 'd', 'r', 'h', 'd']
['p', 'a', ' ', 'o', ' ']
['p', 'y', 'f', 'p', 'y']
['y', ' ', 'r', 'e', 'e']
[' ', 'm', 'i', ' ', 'a']
['b', 'y', 'e', 'a', 'r']
['i', ' ', 'n', ' ', ' ']
['r', 'd', 'd', 'g', 'a']
['t', 'e', '.', 'o', 'h']
y= 
 ['h', 'a', ' ', 'o', 'e']

```

In [12]:
pred_num = 25
xin = []
y=[]
for i in range(pred_num):
  x=[]
  for j in range(0, len(total_index)-1-pred_num, pred_num):  
    x.append(total_index[j+i])
  xin.append(x)
  #print(x)
for j in range(0, len(total_index)-1-pred_num, pred_num):  
  y.append(total_index[j+pred_num])
#print('y=','\n',y)

In [16]:
len(xin),len(xin[0])

(25, 24035)

we can probably use reshape function to do this thing.

In [18]:
len(y),y[:20]

(24035,
 [44, 58, 68, 62, 73, 8, 67, 65, 2, 61, 62, 75, 58, 2, 54, 73, 2, 57, 58, 73])

In [19]:
for each in xin:
  print(len(each))

24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035
24035


We are removing the last 2 characters to keep the length of each array equal

In [0]:
#X = [np.stack(xin[i][:-2]) for i in range(pred_num)]
#Y = np.stack(y[:-2])

In [0]:
#X

In [0]:
#Y[:8]

In [0]:
#X[0].shape, Y.shape

In [20]:
hidden_layers = 256
vocab_size = 86
n_fac = 42

Creating a simple RNN

In [21]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, activation='relu'),
        Dense(vocab_size, activation='softmax')
    ])

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 42)            3612      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_1 (Dense)              (None, 86)                22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [24]:
model.fit(np.stack(xin, 1), y, batch_size=128, epochs=100)

Epoch 1/100
24035/24035 [==============================] - 7s 305us/step - loss: 3.1870
Epoch 2/100
24035/24035 [==============================] - 5s 208us/step - loss: 2.7969
Epoch 3/100
24035/24035 [==============================] - 5s 217us/step - loss: 2.5241
Epoch 4/100
24035/24035 [==============================] - 5s 216us/step - loss: 2.3748
Epoch 5/100
24035/24035 [==============================] - 5s 201us/step - loss: 2.2634
Epoch 6/100
24035/24035 [==============================] - 5s 210us/step - loss: 2.1645
Epoch 7/100
24035/24035 [==============================] - 5s 217us/step - loss: 2.0818
Epoch 8/100
24035/24035 [==============================] - 5s 220us/step - loss: 2.0078
Epoch 9/100
24035/24035 [==============================] - 5s 214us/step - loss: 1.9404
Epoch 10/100
24035/24035 [==============================] - 5s 218us/step - loss: 1.8738
Epoch 11/100
24035/24035 [==============================] - 5s 215us/step - loss: 1.8165
Epoch 12/100
24035/24035 [====

In [25]:
model.save_weights('simpleRNN_3pred.h5')

In [26]:
model.load_weights('simpleRNN_3pred.h5')

In [27]:
model.save_weights('simpleRNN_7pred.h5')

In [28]:
model.load_weights('simpleRNN_7pred.h5')

1. First convert the input to indices
2. Then expand the dimension to match the model's output format
3. Predict the 8th character using the input
4. As we are using softmax activation in the last layer of the model, we get the probability of every 86 characters in our vocabulary. So the character with the maximum probability will be the 8th predicted character by the model

In [29]:
def predict_next_char(inp):
    c=100
    st=''+inp
    while(c):
      c=c-1
      #print(inp)
      index = [char_to_index[i] for i in inp]
      arr = np.expand_dims(np.array(index), axis=0)
      prediction = model.predict(arr)
      prediction = index_to_char[np.argmax(prediction)]
      #print(prediction)
      st=st+prediction
      if prediction=='.':
        return st 
      inp=inp[1:]
      inp=inp+prediction
    return st 

In this example, prediction is being done for the 8th character(pred_num = 7)

In [30]:
predict_next_char('SUPPOSING that Truth is a')

'SUPPOSING that Truth is as look passion--mamake of proving and peorled\nwhich the lofare to smentange, and\nof darive\n"uspernat'